In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tqdm.auto import tqdm
from catboost import CatBoostRegressor
import os
import json
from openai import OpenAI

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)

In [6]:
df = pd.read_csv('data_inner/raw_nhanes_data.csv')
df

/var/folders/fh/jpzybjhs0nq0265pm39s6kf80000gn/T/ipykernel_81721/3216463404.py:1: DtypeWarning: Columns (1427,1428,1435,1436,1445,1446,1546,1547,1579,1580) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_inner/raw_nhanes_data.csv')


,Respondent sequence number,Respondent sequence number that includes an identifier for NHANES III and NHANES continuous,gender,age,Unnamed: 0_x,SDDSRVYR_x,Serum selenium replicate 2 (ug/L),total Lycopene replicate 1 (ug/dL),total Lycopene replicate 2 (ug/dL),Serum alpha carotene replicate 1 (ug/dL),Serum alpha carotene replicate 2 (ug/dL),Serum cis-Beta carotene (ug/dL),Serum beta cryptoxanthin replicate 1 (ug/dL),Serum beta cryptoxanthin replicate 2 (ug/dL),Serum combined lutein/zeaxanthin replicate 1 (ug/dL),Serum combined lutein/zeaxanthin replicate 2 (ug/dL),Vitamin A replicate 1 (ug/dL),Vitamin A replicate 2 (ug/dL),Urinary cadmium replicate 2 (ng/mL),Serum total calcium replicate 1 (mg/dL),Blood lead replicate 1 (ug/dL),Blood lead replicate 2 (ug/dL),Serum potassium replicate 1 (mmol/L),Serum selenium replicate 1 (ug/L),Serum vitamin C replicate 1 (mg/dL),Serum vitamin C replicate 2 (mg/dL),Urinary cadmium replicate 1 (ng/mL),Serum chloride replicate 1 (mmol/L),Serum cotinine replicate 1 (ng/mL),Serum cotinine replicate 2 (ng/mL),Serum cotinine replicate 3 (ng/mL),Urinary iodine replicate 2 (ng/mL),Sodium replicate 1 (mmol/L),Vitamin B12 replicate 2 (pg/mL),Vitamin B12 replicate 1 (pg/mL),Urinary iodine replicate 1 (ng/mL),Acrylamide (pmoL/g Hb),Glycideamide (pmol/g Hb),Decanaldehyde (ng/mL),Propanaldehyde (ng/mL),Butyraldehyde (ng/mL),Pentanaldehyde (ng/mL),Hexanaldehyde (ng/mL),Heptaldehyde (ng/mL),Octanaldehyde (ng/mL),Nonanaldehyde (ng/mL),Benzaldehyde (ng/mL),Crotonaldehyde (ng/mL),Isopentanaldehyde (ng/mL),o-Tolualdehyde (ng/mL),"1-Aminonaphthalene, urine (pg/mL)","2-Aminonaphthalene, urine (pg/mL)","4-Aminobiphenyl, urine (pg/mL)","o-Anisidine, urine (pg/mL)","2,6-Dimethylaniline, urine (pg/mL)","o-Toluidine, urine (pg/mL)","4,4'-Diaminodiphenylmethane (ng/mL)","2,4-Diaminotoluene (ng/mL)","1,5-Diaminonaphthalene (ng/mL)","2,6-Diaminotoluene (ng/mL)",p-Phenylenediamine (ng/mL),Atrazine (ug/L),Atrazine mercapturate (ug/L),Diaminochloroatrazine (ug/L),Desethyl atrazine (ug/L),Desisopropyl atrazine (ug/L),Desisopropyl atrazine mercapturate (ug/L),"2,2',4,4',5,5'-hexabromobiphenyl (pg/g)","2,2',4,4',5,5'-hexabromobiphenyl lipid adj (pg/g)","2,2',4-tribromodiphenyl ether (pg/g)","2,2',4-tribromodiphenyl ether lipid adj (ng/g)","2,4,4'-tribromodiphenyl ether (pg/g)","2,4,4'-tribromodiphenyl ether lipid adj (ng/g)","2,2',4,4'-tetrabromodiphenyl ether (pg/g)","2,2',4,4'-tetrabromodiphenyl ether lipid adj (ng/g)","2,2',3,4,4'-pentabromodiphenyl ether (pg/g)","2,2',3,4,4'-pentabromodiphenyl ether lipid adj (ng/g)","2,2',4,4',5-pentabromodiphenyl ether (pg/g)","2,2',4,4',5-pentabromodiphenyl ether lipid adj (ng/g)","2,2',4,4',6-pentabromodiphenyl ether (pg/g)","2,3',4,4'-tetrabromodiphenyl ether (pg/g)","2,3',4,4'-tetrabromodiphenyl lipid adj (ng/g)","2,2',4,4',6-pentabromodiphenyl lipid adj (ng/g)","2,2',4,4',5,5'-hexabromodiphenyl ether (pg/g)","2,2',4,4',5,5'-hexabromodiphenyl ether lipid adj (ng/g)","2,2',4,4',5,6'-hexabromodiphenyl ether (pg/g)","2,2',4,4',5,6'-hexabromodiphenyl ether lipid adj (ng/g)","2,2',3,4,4',5',6-heptabromodiphenyl ether (pg/g)","2,2',3,4,4',5',6-heptabromodiphenyl ether lipid adj (ng/g)",Blood cadmium (ug/L),Urinary cadmium (ng/mL),Total Calcium (mg/dL),Serum total calcium replicate 2 (mg/dL),Serum chloride (mmol/L),Serum chloride replicate 2 (mmol/L),Cobalt (ug/L),Chromium (ug/L),Serum Copper (ug/dL),Serum cotinine (ng/mL),"Hydroxycotinine, Serum (ng/mL)","Anabasine, urine (ng/mL)","Anatabine, urine (ng/mL)","Total Cotinine, urine (ng/mL)","Cotinine-n-oxide, urine (ng/mL)","Total Hydroxycotinine, urine (ng/mL)","1-(3-Pyridyl)-1-butanol-4-carboxylic acid, urine (ng/mL)","NNAL, urine (ng/mL)","Nicotine, urine (ng/mL)","Nornicotine, urine (ng/mL)","Nicotine-1 N-oxide, urine (ng/mL)","4-hydroxy-4-(3-pyridyl) butanoic acid, urine (ng/mL)",3-(Ethylcarbamoyl)benzoic acid (ug/L),DEET acid (ug/L),DEET (ug/L),Desethyl hydroxy DEET (ug/L),PCB199 (ng/g),PCB28 (ng/g),PCB28 Lipid Adj (ng/g),PCB44 (ng/g),PCB4

In [3]:
system_prompt = """
You are professional translator of medical terms for NHANES dataset from english to russian.  You should provide very accurate and professional translations. 
You are provided with a list of tuples to translate where first element is a key and second is translation. 
Output should be a json-parsable string containing json that contains 2 keys: "translation" and "commentary". Commentary should explain the term in russian language. If  you are not sure for translation or not familiar with the term then as a translation output original value and empty commentary. 
Context: this is biomarkers data from NHANES dataset, so all the provided terms are medical or chemical, and it is measured on the human body mainly with blood analysis. 
For output provid a valid json. Example of output: 
"{
  "0": {
    "translation": "",
    "commentary": ""
  },
  "1": {
    "translation": "",
    "commentary": ""
  }"
"""

In [5]:
import os
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY_oz"))

def translate_columns(df, start_id, end_id):
    cols_tuple = list(zip(range(start_id, end_id), df.columns.values[start_id:end_id]))
    translations_df = pd.DataFrame(cols_tuple, columns=['idx', 'original_name'])
    translations_df['idx'] = translations_df['idx'].astype(str)

    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        # model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"{cols_tuple}"}
        ],
        temperature=0,
        max_tokens=4095,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={"type": "json_object"},
        logit_bias={"1734": -100}
    )

    content = json.loads(response.model_dump_json())['choices'][0]['message']['content']

    try:
        data = json.loads(content)  # Try to parse JSON
        return pd.merge(translations_df, pd.DataFrame(data).T, left_on='idx', right_index=True, how='left')
    except json.JSONDecodeError:
        # If JSON parsing fails, save raw response to text file
        filename = f"data_inner/translations_txt/error_response_{start_id}_{end_id}.txt"
        with open(filename, "w") as f:
            f.write(content)
        return None  # Return None to indicate error


batch_size = 10
max_workers = 20

# Process translations in batches with parallel execution and immediate saving
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for start_id in range(289, len(df.columns), batch_size):
        end_id = min(start_id + batch_size, len(df.columns))
        futures.append(
            executor.submit(translate_columns, df, start_id, end_id)
        )

    # Save results as they become available
    for future in as_completed(futures):
        result = future.result()
        if result is not None:  # Check if translation was successful
            translations_df = result
            end_id = translations_df["idx"].max()
            translations_df.to_csv(f"data_inner/translations/translations_{end_id}.csv")

In [7]:
# concat translations
folder_path = 'data_inner/translations'
all_dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(folder_path, filename)
        dff = pd.read_csv(filepath, index_col=0)
        all_dfs.append(dff)

translated = pd.concat(all_dfs, ignore_index=True)
translated = translated.drop_duplicates(subset='idx')
translated['idx'] = translated['idx'].astype(str)
translated = translated.rename(columns={'original_name':'original_name_from_gpt'})
translated

,idx,original_name_from_gpt,translation,commentary
0,1229,Promyelocytes (percent of 100 cells) replicate 2,Промиелоциты (процент из 100 клеток) повтор 2,"Промиелоциты - это молодые клетки крови, предш..."
1,1230,Red blood cell count,Количество эритроцитов,"Эритроциты - красные кровяные тельца, перенося..."
2,1231,Red blood cell count replicate 1,Количество эритроцитов повтор 1,Повторное измерение количества эритроцитов для...
3,1232,Red blood cell count replicate 2,Количество эритроцитов повтор 2,Второе повторное измерение количества эритроци...
4,1233,Red cell distribution width:SI(fraction),Ширина распределения эритроцитов:SI (доля),Ширина распределения эритроцитов (RDW) - это м...
...,...,...,...,...
1603,1004,Mesh vest used,Использовался сетчатый жилет,Сетчатый жилет может использоваться в медицинс...
1604,1005,Warm-up grade (% incline),Угол наклона при разминке (%),Указывает на угол наклона беговой дорожки или ...
1605,1006,Warm-up speed (miles per hour),Скорость разминки (мили в час),"Скорость, с которой участник исследования долж..."
1606,1007,Estimated maximal oxygen uptake (ml/kg/min).,Оценочная максимальная потребляемость кислород...,"Показатель, отражающий максимальное количество..."


In [8]:
cols_tuple = list(zip(range(0, df.shape[1]), df.columns.values))
translations_df = pd.DataFrame(cols_tuple, columns=['idx', 'original_name'])
translations_df['idx'] = translations_df['idx'].astype(str)
translations_df


,idx,original_name
0,0,Respondent sequence number
1,1,Respondent sequence number that includes an id...
2,2,gender
3,3,age
4,4,Unnamed: 0_x
...,...,...
1622,1622,Serum uric acid replicate 1 (mg/dL)
1623,1623,Serum uric acid replicate 2 (mg/dL)
1624,1624,N-telopeptides (NTx) (nmol BCE)
1625,1625,N-telopeptides (NTx) replicate 1 (nmol BCE)


In [9]:
final_df = pd.merge(translations_df, translated, how='left', on='idx')
final_df.to_csv('data/columns_translations.csv')
final_df

,idx,original_name,original_name_from_gpt,translation,commentary
0,0,Respondent sequence number,Respondent sequence number,Номер респондента,"Уникальный идентификационный номер, присвоенны..."
1,1,Respondent sequence number that includes an id...,Respondent sequence number that includes an id...,"Номер респондента, включающий идентификатор дл...","Уникальный номер, который включает в себя спец..."
2,2,gender,gender,пол,Биологический пол участника исследования.
3,3,age,age,возраст,Возраст участника исследования на момент прове...
4,4,Unnamed: 0_x,Unnamed: 0_x,Unnamed: 0_x,NaN
...,...,...,...,...,...
1622,1622,Serum uric acid replicate 1 (mg/dL),Serum uric acid replicate 1 (mg/dL),"Сывороточная мочевая кислота, повтор 1 (мг/дл)",Концентрация мочевой кислоты в сыворотке крови...
1623,1623,Serum uric acid replicate 2 (mg/dL),Serum uric acid replicate 2 (mg/dL),"Сывороточная мочевая кислота, повтор 2 (мг/дл)",Концентрация мочевой кислоты в сыворотке крови...
1624,1624,N-telopeptides (NTx) (nmol BCE),N-telopeptides (NTx) (nmol BCE),N-телопептиды (NTx) (нмоль BCE),"N-телопептиды - маркеры костного метаболизма, ..."
1625,1625,N-telopeptides (NTx) replicate 1 (nmol BCE),N-telopeptides (NTx) replicate 1 (nmol BCE),"N-телопептиды (NTx), повтор 1 (нмоль BCE)","Первый повтор измерения N-телопептидов, маркер..."


In [33]:
final_df[final_df['original_name'] != final_df['original_name_from_gpt']]

,idx,original_name,original_name_from_gpt,translation,commentary
509,509,Serum vitamin D replicate 1 (nmol/L),NaN,NaN,NaN
510,510,Serum vitamin D replicate 2 (nmol/L),NaN,NaN,NaN
511,511,N-Nitrosodiethylamine (NDEA) (ng/L),NaN,NaN,NaN
512,512,N-Nitrosoethylmethylamine (NMEA) (ng/L),NaN,NaN,NaN
513,513,N-Nitrosomorpholine (NMOR) (ng/L),NaN,NaN,NaN
514,514,N-Nitrosopiperidine (NPIP) (ng/L),NaN,NaN,NaN
515,515,N-Nitrosopyrrolidine (ng/L),NaN,NaN,NaN
516,516,"Blood 2,5-Dimethylfuran (ng/mL)",NaN,NaN,NaN
517,517,"Blood 1,1,1,2-tetrachloroethane (ng/mL)",NaN,NaN,NaN
518,518,Blood Nitromethane (pg/mL),NaN,NaN,NaN
